<a href="https://colab.research.google.com/github/koseii2122/Reddit-Vaccine-Data/blob/main/Doc2Vec_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
del df

In [ ]:
df = pd.read_csv('min5.csv', engine='python', encoding='utf-8', error_bad_lines=False)

In [ ]:
df = df[['id', 'author', 'body_pp_4', 'subreddit', 'title']]

In [ ]:
df.shape

(220069, 5)

In [ ]:
df.dtypes

id           object
author       object
body_pp_4    object
subreddit    object
title        object
dtype: object

In [ ]:
df = df[df['body_pp_4'].notnull()]
df.reset_index(drop=True, inplace=True)

In [ ]:
df['body_pp_4'].apply(lambda x: len(x.split(' '))).sum()

4795313

In [ ]:
# 1,945,104 Unique words
# 4795313

In [ ]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tagged = df.apply(lambda r: TaggedDocument(words=tokenize_text(r['body_pp_4'] + r['title']), tags=[r.id, r.subreddit, r.title, r.author]), axis=1)

In [ ]:
tagged = df.apply(lambda r: TaggedDocument(words=tokenize_text(r['body_pp_4'] + r['title']), tags=[r.id]), axis=1)

In [ ]:
tagged[9]

TaggedDocument(words=['another', 'straw', 'man', 'always', 'instead', 'actually', 'addressing', 'issue', 'its', 'funny', 'go', 'dan', 'stans', 'turn', 'around', 'exact', 'thing', 'scomo'], tags=['hnmpw3z', 'CoronavirusDownunder', ['emails', 'reveal', 'pfizer', 'was', "'enthusiastic", 'to', 'engage', 'with', 'australia', 'about', 'covid-19', 'vaccine'], 'Nath280'])

In [ ]:
model = Doc2Vec(size=200, dbow_words= 1, dm=0, iter=1,  window=3, seed=1337, min_count=5, workers=4,alpha=0.025, min_alpha=0.025)
model.build_vocab(tagged)
for epoch in range(80):
    print("epoch "+str(epoch))
    model.train(tagged, total_examples=len(df), epochs=1)
    model.save('Doc2Vec_vacc_15.model')
    model.alpha -= 0.001  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79


In [ ]:
similars = model.docvecs.most_similar(positive=[model.infer_vector(['conspiracy'])])
print(similars)

[('ParlerWatch', 0.6044207811355591), ('SD_03', 0.03871900960803032), ('h2dxn2z', 0.02791058085858822), ('h9r0f3j', -0.002755356952548027), ('h2ea2bp', -0.0048598796129226685), ('hfjfd8g', -0.0251601729542017), ('mushroompepporoni', -0.030445419251918793), ('hno2fi8', -0.07155387848615646), ('Riot419', -0.15922239422798157), ('hillaryclinternet', -0.31814295053482056)]


In [ ]:
model = Doc2Vec.load('Doc2Vec.model')

FileNotFoundError: ignored

In [ ]:
text = df.body_pp_4.str.split()

In [ ]:
text

0         [course, emails, hunt, resigned, knew, would, ...
1                                           [scomo, fucked]
2         [imagine, lockdown, could, avoided, slime, min...
3         [really, hope, fuck, cost, scomo, lnp, electio...
4         [would, lot, easier, forgive, governments, stu...
                                ...                        
219851    [>searching, positive, male, role, model, life...
219852    [>, travis, character, way, men, struggling, w...
219853                                       [hammer, yeah]
219854    [oh, yes, thanks, noticed, that, never, heard,...
219855                                      [absolutely, :]
Name: body_pp_4, Length: 219856, dtype: object

In [ ]:
w2v_model = gensim.models.Word2Vec(
    window = 4,
    min_count = 2,
    workers = 4
)

In [ ]:
del w2v_model

In [ ]:
w2v_model.build_vocab(text, progress_per = 1000)
w2v_model.epochs = 100

In [ ]:
w2v_model.corpus_count

219856

In [ ]:
w2v_model.train(text, total_examples = w2v_model.corpus_count, epochs = 100)

(447900349, 479531300)

In [ ]:
w2v_model.save('w2v2.model')

In [ ]:
w2v_model.most_similar('complications', topn = 50)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('complication', 0.7471787929534912),
 ('myocarditis', 0.575736403465271),
 ('symptom', 0.5588468313217163),
 ('symptoms', 0.5587630271911621),
 ('hospitalized', 0.5362088680267334),
 ('effects', 0.5315114259719849),
 ('effect', 0.5313609838485718),
 ('illness', 0.5312471389770508),
 ('asthma', 0.5173354148864746),
 ('comorbidity', 0.5168099403381348),
 ('illnesses', 0.5125764608383179),
 ('neurologic', 0.5024263262748718),
 ('severe', 0.49865826964378357),
 ('allergies', 0.4982668459415436),
 ('debilitating', 0.49582797288894653),
 ('injury', 0.490342915058136),
 ('clot', 0.4859618544578552),
 ('adverse', 0.4842502772808075),
 ('side-effects', 0.4842219054698944),
 ('comorbidities', 0.48161065578460693),
 ('pericarditis', 0.4771282374858856),
 ('contraindication', 0.4734051525592804),
 ('neurological', 0.46981269121170044),
 ('conditions', 0.4685911536216736),
 ('condition', 0.46547165513038635),
 ('scarring', 0.4609470069408417),
 ('risk', 0.46071991324424744),
 ('issues', 0.4574918

In [ ]:
w2v_model.wv.most_similar(positive = ["side", "effect"], topn = 30)

[('effects', 0.7395542860031128),
 ('side-effects', 0.6365437507629395),
 ('reaction', 0.6334787011146545),
 ('complication', 0.5998677015304565),
 ('symptom', 0.5759552717208862),
 ('symptoms', 0.5621201992034912),
 ('affects', 0.5605800151824951),
 ('reactions', 0.5554724931716919),
 ('adverse', 0.5505796670913696),
 ('complications', 0.5456244945526123),
 ('"side', 0.5261719226837158),
 ('side-effect', 0.5221983790397644),
 ('myocarditis', 0.5194922685623169),
 ('effects"', 0.5034385919570923),
 ('long-term', 0.49193665385246277),
 ('outcome', 0.4860057830810547),
 ('long-lasting', 0.46791696548461914),
 ('lasting', 0.46281272172927856),
 ('clot', 0.451691210269928),
 ('term', 0.44053229689598083),
 ('long-haul', 0.4384503662586212),
 ('flare', 0.43765556812286377),
 ('mild', 0.4349224865436554),
 ('debilitating', 0.4341469705104828),
 ('affect', 0.4325489401817322),
 ('effects…', 0.4298357367515564),
 ('illness', 0.4282490611076355),
 ('longterm', 0.4258002042770386),
 ('very', 0.4

In [ ]:
w2v_model.wv.most_similar("conspiracy", topn = 50)

[('qanon', 0.6825639009475708),
 ('complicity', 0.663777232170105),
 ('deadman', 0.5861586332321167),
 ('crackpot', 0.5524335503578186),
 ('lab-leak', 0.5412991046905518),
 ('nonsense', 0.5404069423675537),
 ('propaganda', 0.5386761426925659),
 ('conspiracies', 0.5362856984138489),
 ('conspiratorial', 0.5313394069671631),
 ('"conspiracy', 0.5202805995941162),
 ('q', 0.519848644733429),
 ('crazy', 0.5170199871063232),
 ('weirdo', 0.5118709802627563),
 ('theory', 0.5048273205757141),
 ('bullshit', 0.5027326345443726),
 ('belief', 0.49035578966140747),
 ('pseudoscience', 0.4879256784915924),
 ('horseshit', 0.48712944984436035),
 ('anti-vax', 0.48140034079551697),
 ('"necessary"', 0.4686968922615051),
 ('misinformation', 0.460904985666275),
 ('q-anon', 0.45859456062316895),
 ('nutter', 0.45166927576065063),
 ('right-wing', 0.44802847504615784),
 ('origination', 0.4479939937591553),
 ('falsehood', 0.447714626789093),
 ('narrative', 0.44571956992149353),
 ('anti-vaxx', 0.443324476480484),
 (

In [ ]:
w2v_model.wv.most_similar("complications")

[('complication', 0.7970740795135498),
 ('myocarditis', 0.6171822547912598),
 ('side-effects', 0.6135694980621338),
 ('neurological', 0.6042200326919556),
 ('asthma', 0.5935596227645874),
 ('effect', 0.5923874974250793),
 ('effects', 0.5782551765441895),
 ('neurologic', 0.5730143189430237),
 ('symptom', 0.5700149536132812),
 ('illnesses', 0.5650389194488525)]

In [ ]:
w2v_model.wv.most_similar(positive = ["distribution", "bad"], topn = 50)

[('good', 0.555018961429596),
 ('terrible', 0.4945800304412842),
 ('well', 0.45703330636024475),
 ('logistics', 0.45200246572494507),
 ('distributing', 0.4470675587654114),
 ('ramping', 0.44687262177467346),
 ('ramp', 0.43572041392326355),
 ('rollout', 0.4356788396835327),
 ('production', 0.4341040849685669),
 ('fair', 0.42851918935775757),
 ('fast', 0.4267268776893616),
 ('certainly', 0.4260709285736084),
 ('similar', 0.42175519466400146),
 ('manufacturing', 0.421093225479126),
 ('slow', 0.42059358954429626),
 ('supply', 0.416046679019928),
 ('poor', 0.4071093797683716),
 ('better', 0.40677958726882935),
 ('great', 0.40582066774368286),
 ('accelerate', 0.40472567081451416),
 ('much', 0.40447795391082764),
 ('handling', 0.398689866065979),
 ('allocate', 0.3978491425514221),
 ('perfect', 0.3969457149505615),
 ('horrible', 0.396670937538147),
 ('happening', 0.3946995139122009),
 ('logistical', 0.3928965926170349),
 ('improve', 0.3902639150619507),
 ('complicate', 0.38901832699775696),
 (

In [ ]:
w2v_model.wv.most_similar(positive = ["vaccine"], negative = ["india"])

[('vax', 0.5646072030067444),
 ('shot', 0.5167040824890137),
 ('vaccines', 0.49471163749694824),
 ('jab', 0.4817871153354645),
 ('vaccination', 0.4740978479385376),
 ('shots', 0.47020137310028076),
 ('either', 0.4646610617637634),
 ('vaccinated', 0.4590862989425659),
 ('whether', 0.44043639302253723),
 ('vaccine"', 0.42853423953056335)]

In [ ]:
w2v_model.wv.most_similar("omicron", topn = 50)

[('variant', 0.7861006259918213),
 ('variants', 0.7181830406188965),
 ('delta', 0.6915205717086792),
 ('strain', 0.6353638172149658),
 ('antibody', 0.610032856464386),
 ('alpha', 0.57990562915802),
 ('varient', 0.570019006729126),
 ('strains', 0.567523717880249),
 ('b1351', 0.5600037574768066),
 ('mutation', 0.5455003380775452),
 ('d614g', 0.5448943376541138),
 ('neutralizing', 0.5435523390769958),
 ('neutralization', 0.5431193709373474),
 ('mutations', 0.5429086685180664),
 ('booster', 0.5261648893356323),
 ('voc', 0.5234107375144958),
 ('mutant', 0.5203320384025574),
 ('p1', 0.5171103477478027),
 ('infection', 0.5133106708526611),
 ('transmissible', 0.5126519203186035),
 ('effectiveness', 0.5120997428894043),
 ('beta', 0.5119048357009888),
 ('antibodies', 0.5111491084098816),
 ('sars-cov-2', 0.506589949131012),
 ('j&j', 0.5064763426780701),
 ('neutralizes', 0.502124011516571),
 ('kappa', 0.500365138053894),
 ('omnicron', 0.5000152587890625),
 ('sa', 0.4980686604976654),
 ('b117', 0.4

In [ ]:
w2v_model.wv.most_similar("mandate")

[('mandates', 0.8205986618995667),
 ('mandated', 0.6724334359169006),
 ('mandating', 0.6711333394050598),
 ('mandatory', 0.6360436677932739),
 ('policy', 0.6352285742759705),
 ('enforce', 0.6313526630401611),
 ('requirement', 0.6021889448165894),
 ('passports', 0.5926445722579956),
 ('requiring', 0.571552038192749),
 ('osha', 0.5547534823417664)]

In [ ]:
w2v_model.wv.most_similar("antivax", topn = 30)

[('anti-vax', 0.821255624294281),
 ('anti-vaxx', 0.757702112197876),
 ('antivaxx', 0.642516016960144),
 ('anti-vaccine', 0.637732744216919),
 ('anti-vaxxers', 0.6025864481925964),
 ('q', 0.5716720819473267),
 ('anti-vaxxer', 0.560082733631134),
 ('anti-mask', 0.5485765337944031),
 ('anti-science', 0.5483241081237793),
 ('antivaxxers', 0.5411434769630432),
 ('spews', 0.5315593481063843),
 ('pro-vax', 0.52574622631073),
 ('anti', 0.5236144065856934),
 ('trumper', 0.5059484839439392),
 ('antivaxxer', 0.5037360787391663),
 ('facebook', 0.4979485273361206),
 ('idiot', 0.4902229309082031),
 ('pro-trump', 0.48983755707740784),
 ('anti-vac', 0.48402783274650574),
 ('cultist', 0.47690558433532715),
 ('qanon', 0.47457754611968994),
 ('fringe', 0.47316431999206543),
 ('hardcore', 0.47185298800468445),
 ('adamantly', 0.46890750527381897),
 ('q-anon', 0.4687976539134979),
 ('brainwashed', 0.4567398130893707),
 ('conspiratorial', 0.4564148187637329),
 ('religious', 0.4552445709705353),
 ('vocal', 0.

In [ ]:
w2v_model.wv.most_similar("death", topn = 30)

[('deaths', 0.7965793013572693),
 ('mortality', 0.7120257616043091),
 ('hospitalization', 0.6523573398590088),
 ('fatality', 0.635346531867981),
 ('case', 0.6069298982620239),
 ('dying', 0.5994061231613159),
 ('hospitalisation', 0.5902340412139893),
 ('incidence', 0.5683985948562622),
 ('hospitalizations', 0.5622308850288391),
 ('hospitalized', 0.5361501574516296),
 ('infection', 0.5323406457901001),
 ('cases', 0.5182589292526245),
 ('myocarditis', 0.508224368095398),
 ('deaths**', 0.504592776298523),
 ('died', 0.4966011643409729),
 ('0-18', 0.49401283264160156),
 ('reinfection', 0.4928533434867859),
 ('781', 0.4883345067501068),
 ('infections', 0.48816201090812683),
 ('deaths"', 0.4868641197681427),
 ('suffering', 0.48555904626846313),
 ('casualty', 0.4783424139022827),
 ('with-covid', 0.4761310815811157),
 ('complication', 0.47089073061943054),
 ('icu', 0.47015589475631714),
 ('dieing', 0.4676567018032074),
 ('lethality', 0.46499937772750854),
 ('miscarriage', 0.45536500215530396),
 